# <font color='red'>Tratando os dados do Youtube</font>

## <font color='red'>1. Importando as bibliotecas</font>

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import json
import os

sns.set_context("talk", font_scale=1)
sns.set_style("darkgrid")
sns.set_palette('tab10')
sns.set_color_codes()

## <font color='red'>2. Importando e limpando as bases de dados</font>

### 2.1 Importando as inscrições em canais do Youtube

In [2]:
openfile = open(r'/home/mchll/my_project_dir/my_project_env/preparacao_projeto_5/dados/youtube/inscricoes/inscrições.json', encoding='UTF-8')
json_subscriptions = json.load(openfile)

In [3]:
json_subscriptions

[{'contentDetails': {'activityType': 'all',
   'newItemCount': 0,
   'totalItemCount': 354},
  'etag': 'b9Ja1-GCUXVnojrkg-dtKL-_I0E',
  'id': 'rLfJEEByP-rX_879a95TRbdC2fZyg3JumXLAYNrDAHo',
  'kind': 'youtube#subscription',
  'snippet': {'channelId': 'UCQRUzuJ8a5KOhF6jGzdHylw',
   'description': 'A +1 Filmes, sediada em Salvador, é uma produtora de conteúdo audiovisual que nasceu em 2010. Já teve seus filmes exibidos no Canal Brasil, AMC, Sundace TV, TVE-Ba e em diversos festivais nacionais. Com os realizadores Deo e Moara Rocha, desenvolve produtos audiovisuais com foco no mercado independente. \n\nDesde sua criação, a empresa trabalha produção de conteúdos para web.  As produções da +1 Filmes  alcançaram mais de 100 milhões de visualizações.\n\nAlém da vasta produção para web, a empresa realizou dezenas de outras obras, como curtas de ficção para público adulto e infantil, documentários, clipes musicais e webséries, em diversos formatos.',
   'publishedAt': '2015-06-28T02:32:43.000Z',

In [4]:

def extract_columns_dict(json, key1, key2):
    data = []
    for i in json:
        for key, value in i[key1].items():
            if key == key2:
                data.append(value)
    return data

In [5]:
channel = extract_columns_dict(json_subscriptions,'snippet', 'title')
description = extract_columns_dict(json_subscriptions,'snippet', 'description')

df_subscriptions = pd.DataFrame([channel, description]).T
df_subscriptions.columns = ['Channel', 'Description']

df_subscriptions.head()

,Channel,Description
0,+1! Filmes,"A +1 Filmes, sediada em Salvador, é uma produt..."
1,- Síntese -,Contato da produção:\nsintese.contato@gmail.co...
2,1Kilo Oficial,Bem-vindos ao canal oficial da 1Kilo no YouTub...
3,3Blue1Brown,"3Blue1Brown, by Grant Sanderson, is some combi..."
4,3DM,Nesse canal você vai encontrar tutoriais sobre...


In [6]:
df_subscriptions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 739 entries, 0 to 738
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Channel      739 non-null    object
 1   Description  739 non-null    object
dtypes: object(2)
memory usage: 11.7+ KB


### 2.1.1. Salvando os dados tratados

In [7]:
#df_subscriptions.to_csv('./dados_tratados/youtube/subscriptions.csv', index = None)

### 2.2 Importando o histórico de visualizações do Youtube

In [8]:
openfile = open(r'/home/mchll/my_project_dir/my_project_env/preparacao_projeto_5/dados/youtube/historico/histórico-de-visualização.json', encoding='UTF-8')
json_history = json.load(openfile)

In [9]:
def extract_columns(json, key):
    data = []
    for i in json:
        data.append(i[key])
    return data

In [10]:
def extract_columns_2(json, key1, key2):
    data = []
    for i in json:
        if key1 in i:
            if len(i[key1]) > 1: data.pop()
            for key in i[key1]:
                data.append(key[key2])
        else:
            data.append('Advertisement (Google AdSense)')
    return data

In [11]:
video = extract_columns(json_history, 'title')
time = extract_columns(json_history, 'time')

channel = extract_columns_2(json_history, 'subtitles','name')

df_history = pd.DataFrame(data = [video, channel, time]).T
df_history.columns=['Video', 'Channel', 'Time']

In [12]:
df_history.head()

,Video,Channel,Time
0,Watched BAIXANDO DADOS DO YOUTUBE USANDO API -...,O Bruno Germano,2021-07-21T04:04:09.484Z
1,Watched Consumindo dados da API do YOUTUBE com...,Programação Dinâmica,2021-07-21T03:24:33.848Z
2,Watched YouTube Data API Tutorial with Python ...,Python Engineer,2021-07-21T03:10:59.025Z
3,Watched Mayí e Malcolm VL - REAI$ (prod. Coyot...,Mayí,2021-07-20T14:29:49.792Z
4,Watched ACADEMY SPACE - Live Especial com Marc...,SpaceToday,2021-07-19T23:01:37.986Z


In [13]:
df_history['Time'] = df_history['Time'].astype(np.datetime64)

In [14]:
df_history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1217 entries, 0 to 1216
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Video    1217 non-null   object        
 1   Channel  1217 non-null   object        
 2   Time     1217 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 28.6+ KB


In [15]:
df_history.head()

,Video,Channel,Time
0,Watched BAIXANDO DADOS DO YOUTUBE USANDO API -...,O Bruno Germano,2021-07-21 04:04:09.484
1,Watched Consumindo dados da API do YOUTUBE com...,Programação Dinâmica,2021-07-21 03:24:33.848
2,Watched YouTube Data API Tutorial with Python ...,Python Engineer,2021-07-21 03:10:59.025
3,Watched Mayí e Malcolm VL - REAI$ (prod. Coyot...,Mayí,2021-07-20 14:29:49.792
4,Watched ACADEMY SPACE - Live Especial com Marc...,SpaceToday,2021-07-19 23:01:37.986


In [16]:
# Vou remover os milissegundos
df_history['Time'] = df_history['Time'].astype(str).str[:-4]

In [17]:
# Checando se deu certo
df_history.head(5)

,Video,Channel,Time
0,Watched BAIXANDO DADOS DO YOUTUBE USANDO API -...,O Bruno Germano,2021-07-21 04:04:09
1,Watched Consumindo dados da API do YOUTUBE com...,Programação Dinâmica,2021-07-21 03:24:33
2,Watched YouTube Data API Tutorial with Python ...,Python Engineer,2021-07-21 03:10:59
3,Watched Mayí e Malcolm VL - REAI$ (prod. Coyot...,Mayí,2021-07-20 14:29:49
4,Watched ACADEMY SPACE - Live Especial com Marc...,SpaceToday,2021-07-19 23:01:37


In [18]:
df_history['Time'] = df_history['Time'].astype(np.datetime64)

In [19]:
df_history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1217 entries, 0 to 1216
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Video    1217 non-null   object        
 1   Channel  1217 non-null   object        
 2   Time     1217 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 28.6+ KB


In [20]:
df_history.head()

,Video,Channel,Time
0,Watched BAIXANDO DADOS DO YOUTUBE USANDO API -...,O Bruno Germano,2021-07-21 04:04:09
1,Watched Consumindo dados da API do YOUTUBE com...,Programação Dinâmica,2021-07-21 03:24:33
2,Watched YouTube Data API Tutorial with Python ...,Python Engineer,2021-07-21 03:10:59
3,Watched Mayí e Malcolm VL - REAI$ (prod. Coyot...,Mayí,2021-07-20 14:29:49
4,Watched ACADEMY SPACE - Live Especial com Marc...,SpaceToday,2021-07-19 23:01:37


**Eliminando a palavra "Watched" do nome dos vídeos assistidos**

In [21]:
df_history['Video'] = df_history['Video'].str.replace('Watched', '')

In [22]:
df_history.head()

,Video,Channel,Time
0,BAIXANDO DADOS DO YOUTUBE USANDO API - DIÁRIO...,O Bruno Germano,2021-07-21 04:04:09
1,Consumindo dados da API do YOUTUBE com Python...,Programação Dinâmica,2021-07-21 03:24:33
2,YouTube Data API Tutorial with Python - Analy...,Python Engineer,2021-07-21 03:10:59
3,Mayí e Malcolm VL - REAI$ (prod. Coyote Beatz),Mayí,2021-07-20 14:29:49
4,ACADEMY SPACE - Live Especial com Marcelo Gle...,SpaceToday,2021-07-19 23:01:37


**Agora vou separar algumas informações contidas no timestamp e criar novas colunas     
com ano, mês, dia do mês, dia da semana, data e hora:**

In [23]:
# Ano
df_history['Year'] = df_history['Time'].dt.year

# Mês
df_history['Month'] = df_history['Time'].dt.month_name()

# Dia da semana
df_history['Day_week'] = df_history['Time'].dt.day_name()

# Data
df_history['Date'] = df_history['Time'].dt.date

# Hora
df_history['Hour'] = df_history['Time'].dt.hour

df_history.head()

,Video,Channel,Time,Year,Month,Day_week,Date,Hour
0,BAIXANDO DADOS DO YOUTUBE USANDO API - DIÁRIO...,O Bruno Germano,2021-07-21 04:04:09,2021,July,Wednesday,2021-07-21,4
1,Consumindo dados da API do YOUTUBE com Python...,Programação Dinâmica,2021-07-21 03:24:33,2021,July,Wednesday,2021-07-21,3
2,YouTube Data API Tutorial with Python - Analy...,Python Engineer,2021-07-21 03:10:59,2021,July,Wednesday,2021-07-21,3
3,Mayí e Malcolm VL - REAI$ (prod. Coyote Beatz),Mayí,2021-07-20 14:29:49,2021,July,Tuesday,2021-07-20,14
4,ACADEMY SPACE - Live Especial com Marcelo Gle...,SpaceToday,2021-07-19 23:01:37,2021,July,Monday,2021-07-19,23


**Agora vou criar colunas específicas para o momento do dia, exemplo:         
1.Início da manhã,      
2.Manhã,       
3.Tarde,           
4.Noite,    
5.Final da Noite e       
6.Madrugada**     

In [24]:
# Criando uma função e estabelecendo as faixas de horário que pretendo segmentar:
def segmenting_day(x):
    if (x > 4) and (x <= 8):
        return 'Early Morning'
    elif (x > 8) and (x <= 12 ):
        return 'Morning'
    elif (x > 12) and (x <= 16):
        return'Afternoon'
    elif (x > 16) and (x <= 20) :
        return 'Evening'
    elif (x > 20) and (x <= 24):
        return'Night'
    elif (x <= 4):
        return'Late Night'

In [25]:
# Criando uma coluna com a parte do dia e aplicando a função à coluna de horário:
df_history['Part_of_day'] = df_history['Hour'].apply(segmenting_day)

In [26]:
# Verificando se as faixas de horário estão segmentadas:
df_history.head(15)

,Video,Channel,Time,Year,Month,Day_week,Date,Hour,Part_of_day
0,BAIXANDO DADOS DO YOUTUBE USANDO API - DIÁRIO...,O Bruno Germano,2021-07-21 04:04:09,2021,July,Wednesday,2021-07-21,4,Late Night
1,Consumindo dados da API do YOUTUBE com Python...,Programação Dinâmica,2021-07-21 03:24:33,2021,July,Wednesday,2021-07-21,3,Late Night
2,YouTube Data API Tutorial with Python - Analy...,Python Engineer,2021-07-21 03:10:59,2021,July,Wednesday,2021-07-21,3,Late Night
3,Mayí e Malcolm VL - REAI$ (prod. Coyote Beatz),Mayí,2021-07-20 14:29:49,2021,July,Tuesday,2021-07-20,14,Afternoon
4,ACADEMY SPACE - Live Especial com Marcelo Gle...,SpaceToday,2021-07-19 23:01:37,2021,July,Monday,2021-07-19,23,Night
5,[LIVE] Raio-X Emocional da Criança! | Isa Min...,Isa Minatel,2021-07-17 15:24:41,2021,July,Saturday,2021-07-17,15,Afternoon
6,E quando a criança bate do nada? - Isa Minate...,Isa Minatel,2021-07-17 15:15:39,2021,July,Saturday,2021-07-17,15,Afternoon
7,Crianças que gritam demais,Marcos Meier,2021-07-17 15:14:59,2021,July,Saturday,2021-07-17,15,Afternoon
8,CRIANÇA QUE GRITA DEMAIS - 11 DICAS PARA RESO...,Macetes de Mãe,2021-07-17 15:00:17,2021,July,Saturday,2021-07-17,15,Afternoon
9,Introdução à Big Data e Engenharia de Dados ...,Escola Livre de Inteligência Artificial,2021-07-15 00:21:09,2021,July,Thursday,2021-07-15,0,Late Night


### 2.2.1. Salvando os dados tratados

In [27]:
#df_history.to_csv('./dados_tratados/youtube/history.csv', index = None)

### 2.3. Importando o histórico de pesquisa no Youtube

In [28]:
df_search = pd.read_json("./dados/youtube/historico/histórico_de_pesquisa.json", encoding = 'UTF-8')
df_search

,header,title,titleUrl,time,products,details
0,YouTube,Searched for baixando dados do youtube api python,https://www.youtube.com/results?search_query=b...,2021-07-21T04:07:07.998Z,[YouTube],NaN
1,YouTube,Searched for cnn tonight,https://www.youtube.com/results?search_query=c...,2021-07-18T00:30:42.883Z,[YouTube],NaN
2,YouTube,Searched for djonga,https://www.youtube.com/results?search_query=d...,2021-07-03T16:35:13.404Z,[YouTube],NaN
3,YouTube,Searched for desenho animado whindersson nunes,https://www.youtube.com/results?search_query=d...,2021-06-20T23:04:47.445Z,[YouTube],NaN
4,YouTube,Searched for bonel dados,https://www.youtube.com/results?search_query=b...,2021-06-19T12:11:47.091Z,[YouTube],NaN
...,...,...,...,...,...,...
2744,YouTube,Searched for anos 80 musicas internacionais,https://www.youtube.com/results?search_query=a...,2012-12-07T22:43:23.531Z,[YouTube],NaN
2745,YouTube,Searched for dragon ball gt dublado conpleto,https://www.youtube.com/results?search_query=d...,2012-08-04T23:56:47.416Z,[YouTube],NaN
2746,YouTube,Searched for dragon ball gt dublado,https://www.youtube.com/results?search_query=d...,2012-08-04T23:56:16.092Z,[YouTube],NaN
2747,YouTube,Searched for santo antônio de pádua dublado,https://www.youtube.com/results?search_query=s...,2012-06-29T09:57:54.255Z,[YouTube],NaN


In [29]:
# Excluindo colunas que não vou utilizar
df_search.drop(columns=['header', 'titleUrl','products', 'details'], inplace=True)

In [30]:
df_search.head()

,title,time
0,Searched for baixando dados do youtube api python,2021-07-21T04:07:07.998Z
1,Searched for cnn tonight,2021-07-18T00:30:42.883Z
2,Searched for djonga,2021-07-03T16:35:13.404Z
3,Searched for desenho animado whindersson nunes,2021-06-20T23:04:47.445Z
4,Searched for bonel dados,2021-06-19T12:11:47.091Z


In [31]:
df_search['time'] = df_search['time'].astype(np.datetime64)

In [32]:
df_search.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2749 entries, 0 to 2748
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   title   2749 non-null   object        
 1   time    2749 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 43.1+ KB


In [33]:
df_search.head()

,title,time
0,Searched for baixando dados do youtube api python,2021-07-21 04:07:07.998
1,Searched for cnn tonight,2021-07-18 00:30:42.883
2,Searched for djonga,2021-07-03 16:35:13.404
3,Searched for desenho animado whindersson nunes,2021-06-20 23:04:47.445
4,Searched for bonel dados,2021-06-19 12:11:47.091


In [34]:
# Vou remover os milissegundos
df_search['time'] = df_search['time'].astype(str).str[:-4]

In [35]:
df_search.head()

,title,time
0,Searched for baixando dados do youtube api python,2021-07-21 04:07:07
1,Searched for cnn tonight,2021-07-18 00:30:42
2,Searched for djonga,2021-07-03 16:35:13
3,Searched for desenho animado whindersson nunes,2021-06-20 23:04:47
4,Searched for bonel dados,2021-06-19 12:11:47


In [36]:
df_search['time'] = df_search['time'].astype(np.datetime64)

In [37]:
df_search.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2749 entries, 0 to 2748
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   title   2749 non-null   object        
 1   time    2749 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 43.1+ KB


In [38]:
df_search.head()

,title,time
0,Searched for baixando dados do youtube api python,2021-07-21 04:07:07
1,Searched for cnn tonight,2021-07-18 00:30:42
2,Searched for djonga,2021-07-03 16:35:13
3,Searched for desenho animado whindersson nunes,2021-06-20 23:04:47
4,Searched for bonel dados,2021-06-19 12:11:47


**Eliminando a palavra "Searched" do histórico das pesquisas feitas**

In [39]:
df_search['title'] = df_search['title'].str.replace('Searched for', '')

In [40]:
df_search.head()

,title,time
0,baixando dados do youtube api python,2021-07-21 04:07:07
1,cnn tonight,2021-07-18 00:30:42
2,djonga,2021-07-03 16:35:13
3,desenho animado whindersson nunes,2021-06-20 23:04:47
4,bonel dados,2021-06-19 12:11:47


**Agora vou separar algumas informações contidas no timestamp e criar novas colunas     
com ano, mês, dia do mês, dia da semana, data e hora:**

In [41]:
# Ano
df_search['year'] = df_search['time'].dt.year

# Mês
df_search['month'] = df_search['time'].dt.month_name()

# Dia da semana
df_search['day_week'] = df_search['time'].dt.day_name()

# Data
df_search['date'] = df_search['time'].dt.date

# Hora
df_search['hour'] = df_search['time'].dt.hour

df_search.head()

,title,time,year,month,day_week,date,hour
0,baixando dados do youtube api python,2021-07-21 04:07:07,2021,July,Wednesday,2021-07-21,4
1,cnn tonight,2021-07-18 00:30:42,2021,July,Sunday,2021-07-18,0
2,djonga,2021-07-03 16:35:13,2021,July,Saturday,2021-07-03,16
3,desenho animado whindersson nunes,2021-06-20 23:04:47,2021,June,Sunday,2021-06-20,23
4,bonel dados,2021-06-19 12:11:47,2021,June,Saturday,2021-06-19,12


### 2.3.1 Salvando os dados tratados

In [42]:
#df_search.to_csv('./dados_tratados/youtube/search.csv', index = None)

### 2.4. Importando o histórico de visualizações no Youtube Kids

In [43]:
openfile = open(r'/home/mchll/my_project_dir/my_project_env/preparacao_projeto_5/dados/youtube/criancas/histórico_de_visualização_kids.json', encoding='UTF-8')
json_history_kids = json.load(openfile)

In [44]:
video = extract_columns(json_history_kids, 'title')
time = extract_columns(json_history_kids, 'time')

channel = extract_columns_2(json_history_kids, 'subtitles','name')

df_history_kids = pd.DataFrame(data = [video, channel, time]).T
df_history_kids.columns=['Video', 'Channel', 'Time']

In [45]:
df_history_kids.head()

,Video,Channel,Time
0,Watched O convite | Gumball | Mês do Riso | Ca...,Cartoon Network Brasil,2019-12-14T00:29:01.188Z
1,Watched Caillou em Português ★ Episódios Compl...,Caillou em Português - WildBrain,2019-12-14T00:26:37.182Z
2,Watched Caillou em Português | Caillou e a Fad...,Caillou em Português - WildBrain,2019-12-14T00:25:29.935Z
3,Watched https://www.youtube.com/watch?v=PALYEc...,Advertisement (Google AdSense),2019-08-25T18:32:41.842Z
4,Watched CINDERELA - Uma Nova Cindebela. Histor...,Bela Bagunça,2019-08-25T18:24:27.832Z


In [46]:
df_history_kids['Time'] = df_history_kids['Time'].astype(np.datetime64)

In [47]:
df_history_kids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Video    123 non-null    object        
 1   Channel  123 non-null    object        
 2   Time     123 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 3.0+ KB


In [48]:
df_history_kids.head()

,Video,Channel,Time
0,Watched O convite | Gumball | Mês do Riso | Ca...,Cartoon Network Brasil,2019-12-14 00:29:01.188
1,Watched Caillou em Português ★ Episódios Compl...,Caillou em Português - WildBrain,2019-12-14 00:26:37.182
2,Watched Caillou em Português | Caillou e a Fad...,Caillou em Português - WildBrain,2019-12-14 00:25:29.935
3,Watched https://www.youtube.com/watch?v=PALYEc...,Advertisement (Google AdSense),2019-08-25 18:32:41.842
4,Watched CINDERELA - Uma Nova Cindebela. Histor...,Bela Bagunça,2019-08-25 18:24:27.832


In [49]:
# Vou remover os milissegundos
df_history_kids['Time'] = df_history_kids['Time'].astype(str).str[:-4]

In [50]:
df_history_kids.head()

,Video,Channel,Time
0,Watched O convite | Gumball | Mês do Riso | Ca...,Cartoon Network Brasil,2019-12-14 00:29:01
1,Watched Caillou em Português ★ Episódios Compl...,Caillou em Português - WildBrain,2019-12-14 00:26:37
2,Watched Caillou em Português | Caillou e a Fad...,Caillou em Português - WildBrain,2019-12-14 00:25:29
3,Watched https://www.youtube.com/watch?v=PALYEc...,Advertisement (Google AdSense),2019-08-25 18:32:41
4,Watched CINDERELA - Uma Nova Cindebela. Histor...,Bela Bagunça,2019-08-25 18:24:27


In [51]:
df_history_kids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Video    123 non-null    object
 1   Channel  123 non-null    object
 2   Time     123 non-null    object
dtypes: object(3)
memory usage: 3.0+ KB


In [52]:
df_history_kids['Time'] = df_history_kids['Time'].astype(np.datetime64)

In [53]:
df_history_kids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Video    123 non-null    object        
 1   Channel  123 non-null    object        
 2   Time     123 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 3.0+ KB


In [54]:
df_history_kids.head()

,Video,Channel,Time
0,Watched O convite | Gumball | Mês do Riso | Ca...,Cartoon Network Brasil,2019-12-14 00:29:01
1,Watched Caillou em Português ★ Episódios Compl...,Caillou em Português - WildBrain,2019-12-14 00:26:37
2,Watched Caillou em Português | Caillou e a Fad...,Caillou em Português - WildBrain,2019-12-14 00:25:29
3,Watched https://www.youtube.com/watch?v=PALYEc...,Advertisement (Google AdSense),2019-08-25 18:32:41
4,Watched CINDERELA - Uma Nova Cindebela. Histor...,Bela Bagunça,2019-08-25 18:24:27


**Eliminando a palavra "Watched" do nome dos vídeos assistidos**

In [55]:
df_history_kids['Video'] = df_history_kids['Video'].str.replace('Watched', '')

In [56]:
df_history_kids.head()

,Video,Channel,Time
0,O convite | Gumball | Mês do Riso | Cartoon N...,Cartoon Network Brasil,2019-12-14 00:29:01
1,Caillou em Português ★ Episódios Completos ★ ...,Caillou em Português - WildBrain,2019-12-14 00:26:37
2,Caillou em Português | Caillou e a Fada do De...,Caillou em Português - WildBrain,2019-12-14 00:25:29
3,https://www.youtube.com/watch?v=PALYEcVINRc,Advertisement (Google AdSense),2019-08-25 18:32:41
4,CINDERELA - Uma Nova Cindebela. Historinha da...,Bela Bagunça,2019-08-25 18:24:27


**Agora vou separar algumas informações contidas no timestamp e criar novas colunas     
com ano, mês, dia do mês, dia da semana, data e hora:**

In [57]:
# Ano
df_history_kids['Year'] = df_history_kids['Time'].dt.year

# Mês
df_history_kids['Month'] = df_history_kids['Time'].dt.month_name()

# Dia da semana
df_history_kids['Day_week'] = df_history_kids['Time'].dt.day_name()

# Data
df_history_kids['Date'] = df_history_kids['Time'].dt.date

# Hora
df_history_kids['Hour'] = df_history_kids['Time'].dt.hour

df_history_kids.head()

,Video,Channel,Time,Year,Month,Day_week,Date,Hour
0,O convite | Gumball | Mês do Riso | Cartoon N...,Cartoon Network Brasil,2019-12-14 00:29:01,2019,December,Saturday,2019-12-14,0
1,Caillou em Português ★ Episódios Completos ★ ...,Caillou em Português - WildBrain,2019-12-14 00:26:37,2019,December,Saturday,2019-12-14,0
2,Caillou em Português | Caillou e a Fada do De...,Caillou em Português - WildBrain,2019-12-14 00:25:29,2019,December,Saturday,2019-12-14,0
3,https://www.youtube.com/watch?v=PALYEcVINRc,Advertisement (Google AdSense),2019-08-25 18:32:41,2019,August,Sunday,2019-08-25,18
4,CINDERELA - Uma Nova Cindebela. Historinha da...,Bela Bagunça,2019-08-25 18:24:27,2019,August,Sunday,2019-08-25,18


### 2.4.1. Salvando os dados tratados

In [58]:
#df_history_kids.to_csv('./dados_tratados/youtube/history_kids.csv', index = None)

## <font color='red'>Fim</font>